<!--NOTEBOOK_HEADER-->
*This notebook contains material from [CBE60499](https://ndcbe.github.io/CBE60499);
content is available [on Github](git@github.com:ndcbe/CBE60499.git).*


<!--NAVIGATION-->
< [1.1 60 Minutes to Pyomo: An Energy Storage Model Predictive Control Example](https://ndcbe.github.io/CBE60499/01.01-Pyomo-Nuts-and-Bolts.html) | [Contents](toc.html) | [Tag Index](tag_index.html) | [2.0 Optimization Modeling with Applications](https://ndcbe.github.io/CBE60499/02.00-Optimization-Modeling.html) ><p><a href="https://colab.research.google.com/github/ndcbe/CBE60499/blob/master/docs/01.02-Pyomo-Mini-Project.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://ndcbe.github.io/CBE60499/01.02-Pyomo-Mini-Project.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

In [ ]:
# IMPORT DATA FILES USED BY THIS NOTEBOOK
import os,  requests

file_links = [("data/Prices_DAM_ALTA2G_7_B1.csv", "https://ndcbe.github.io/CBE60499/data/Prices_DAM_ALTA2G_7_B1.csv")]

# This cell has been added by nbpages. Run this cell to download data files required for this notebook.

for filepath, fileurl in file_links:
    stem, filename = os.path.split(filepath)
    if stem:
        if not os.path.exists(stem):
            os.mkdir(stem)
    if not os.path.isfile(filepath):
        with open(filepath, 'wb') as f:
            response = requests.get(fileurl)
            f.write(response.content)


# 1.2 Pyomo Mini-Project: Receding Horizon Stochastic Control

**Deadline:** Friday, March 5, 2021

## 1.2.1 Assignment Goals

The **primary purpose** of this assignment is to give you **hands-on-keyboard experience** with optimization modeling and analysis. In this assignment, I incorporated many of the best practices I share with my Ph.D. students during their first year using Pyomo for research. Specifically:
* Planning is the most important (and often most overlooked step) in computer programming. Two best practices are to (1) write pseudocode and (2) write the optimization model on paper and perform degree of freedom analysis before you use the computer. We practiced tip (2) in class. If you cannot do (1) and (2), no amount of Googling Pyomo/Python syntax will save you.
* Functions (or classes) allow us to reuse most of our code. Below I gave you function definitions and doc strings to show you a good way to "modularize" this analysis. Once the functions defined below work, performing the analysis is easy.
* Solving your optimization problem a few times is (often) not a sufficient research result. Who cares? Application-driven research is all about the analysis. Optimization is a tool to better answer analysis questions. By giving you >50% of the code for this assignment, I am intentionally emphasizing the analysis (not just coding).  You will see we can take a really simple optimization model and ask/answer some interesting analysis questions.

You should start this assignment early. Almost everyone will make at least one Python/Pyomo syntax mistake. Give yourself plenty of time to ask questions on Slack. Please also be generous with helping your classmates. This assignment (and grades in general in this class) is not a competition. You are welcome (encouraged!) to post screenshots of your code in Slack when asking/answering questions. Please refrain from posting your discussion responses. You are welcome to discuss the analysis questions on Slack; everyone should answer the discussion questions in their own words.

In [ ]:
# This code cell installs packages on Colab

import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_idaes()
    helper.install_ipopt()
    helper.download_data(['Prices_DAM_ALTA2G_7_B1.csv'])

In [ ]:
import pandas as pd
import pyomo.environ as pyo
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ca_data = pd.read_csv('./data/Prices_DAM_ALTA2G_7_B1.csv',names=['price'])

In [ ]:
def prepare_data_array(day=2,num_days=8):
    ''' Prepare numpy array of prices
    
    Arguments:
        day: day to start (0 = January 1, 2015)
        num_days: numbers of days to include in the vector
        
    Returns:
        numpy array of prices
    '''

    return ca_data["price"][(day)*24:24*(day+num_days)].to_numpy()

In the previous notebook, we considered model predictive control (MPC) of a battery energy storage system interacting with the electricity market. In essence, we posed an optimization problem:

*Given a forecasting for energy prices, what charging and discharging strategy maximizes net market revenue?*

In this homework mini-project, you will consider two modifications to make the example more realistic:
1. Add a constraint to prevent simultanous charging and discharging.
1. Receding horizon control
2. Uncertainty

The goal is to help you develop Python and Pyomo programming skills.

## 1.2.2 Model Refinement and Code Reorganization

### 1.2.2.1 Revised Model

Propose a new mathematical model that includes a constraint to prevent charging and discharging. Hint: using either integer variables or a disjunction will ensure the objective and constraints remaining linear.

Below is the model description copied from our in-class example. **Please edit** such that this notebook describes your complete new model.

**Sets**

Timesteps: $\mathcal{T} = \{0, 1, ..., N\}$
    
Timesteps without the initial time: $\mathcal{T}' = \{1, ..., N\}$

**Variables**
* $E_t$, energy stored in battery at time $t$, units: MWh
* $d_t$, battery discharge power (sold to market) during time interval [t-1, t), units: MW
* $c_t$, battery charge power (purchased from the market) during time interval [t-1, t), units: MW

**Parameters**
* $\pi_t$: Energy price during time interval [t-1, t), units: \$/MW
* $\eta$: Round trip efficiency, units: dimensionless
* $c_{max}$ Maximum charge power, units: MW
* $d_{max}$ Maximum discharge power, units: MW
* $E_{max}$ Maximum storage energy, units: MWh
* $E_{0}$ Energy in storage at time $t=0$, units: MWh
* $\Delta t = 1$ hour, Timestep for grid decisions and prices (fixed)

**Optimization Problem**
$$
\begin{align*}
    \max_{\mathbf{E},\mathbf{d},\mathbf{c}} \quad & \psi := \sum_{t \in \mathcal{T}'} \pi_{t} \Delta t (d_{t} - c_{t}) \\
\mathrm{s.t.} \quad & E_{t} = E_{t-1} + \Delta t \left( c_{t} \sqrt{\eta} - \frac{d_{t}}{\sqrt{\eta}} \right), ~~ \forall ~ t \in \mathcal{T}' \\
    & E_{0} = E_{N} \\
    & 0 \leq c_{t} \leq c_{max}, ~~\forall ~ t \in \mathcal{T}' \\
    & 0 \leq d_{t} \leq d_{max}, ~~\forall ~ t \in \mathcal{T}' \\
    & 0 \leq E_{t} \leq E_{max},  ~~\forall ~ t \in \mathcal{T}'
\end{align*}
$$

### 1.2.2.2 Degree of Freedom Analysis

Perform degree of freedom analysis. Identify:
1. Total number of continous variables
2. Total number of integer variables (or disjunctions if using)
3. Total number of equality constraints
4. Total number of inequality constraints
5. Degrees of freedom

Tip: perform this on paper then typeset below.

**Number of continous variables:**

**Number of integer variables:**

**Number of equality constraints:**

**Number of inequality constraints:**
    
**Degrees of freedom.** *Please discuss in a few sentences. There are a few ways to do this; we are less concerned about you getting the "right" answer and more about giving an answer with a supporting argument.*

### 1.2.2.3 Python Code

Programmers organize their code into functions, often following the rule "1 task/action = 1 function". Here are a few of the reasons you should always do this:
1. Functions make your code easier to use
2. More specifically, functions force you to abstract/generalize your code
3. Writing doc strings (required for this course!) forces you to document your code and think about the structure.
4. Functions allow your code to best tested piece by piece

To show you how to do this, we have provided skeletons for several functions in this assignment. Based on the doc string, complete the functions.

In [ ]:
def build_model(price_data, horizon_length, include_pbc=True, include_disjunction=True):
    '''
    Builds HORIZON pyomo model
    
    Inputs:
    price_data: dictionary of price data
    horizon_length: number of timesteps to evaluate
    include_pbc: Boolean, include periodic boundary condition constraint
    
    Returns:
    m: HORIZON pyomo model
    '''
    
    m = pyo.ConcreteModel()
    
    # YOUR SOLUTION HERE
    
    return m


In [ ]:
def extract_solution(m):
    '''
    Function to extract the solution from the solved pyomo model
    
    Inputs:
        m: Solved pyomo model
    
    Outputs:
        c_control: numpy array of charging power values, MW
        d_control: numpy array of discharging power values, MW
        E_control: numpy array of energy stored values, MWh
        t: numpy array of time indices from the Pyomo model, hr
    '''
    
    c_control = np.empty(m.N)
    d_control = np.empty(m.N)
    E_control = np.empty(m.N)
    t = np.empty(m.N)

    # Tip: We added a negative sign to d to help with plotting in the code discussed during class.
    # YOUR SOLUTION HERE
    
    return c_control, d_control, E_control, t

In [ ]:
def plot_solution(c, d, E, t=None):
    '''
    Function to plot the solution of the HORIZON problem
    
    Inputs:
        c_control: numpy array of charging power values, MW
        d_control: numpy array of discharging power values, MW
        E_control: numpy array of energy stored values, MWh
        t: numpy array of time indices from the Pyomo model, hr. (default=None)
    
    Actions. Creates the following plots: 
        State of charge (energy) verus time
        Power to the grid verus time
    '''
    
    # Plot the state of charge (E)
    plt.figure()

    N = len(c)
    assert N > 0, "Input t must have length greate than 0"
    assert len(d) == N, "All inputs should be the same length"
    assert len(E) == N, "All inputs should be the same length"
    
    if t is None:
        # add E0
        t = np.array(range(1,N+1))
        t_ = np.array(range(0,N+1))
    else:
        t_ = np.concatenate(([0],  t))

    # YOUR SOLUTION HERE
    
    # Plot charging/discharging plot
    plt.figure()

    # double up first data point to make the step plot
    c_control_ = np.concatenate(([c[0]],  c))
    d_control_ = np.concatenate(([d[0]],  d))

    plt.step(t_,c_control_,'r.-',where='pre')
    plt.step(t_,d_control_,'g.-',where='pre')
    plt.xlabel('Time (hr)')
    plt.ylabel('Power from Grid (MW)')
    plt.grid(True)
    plt.show()
    
    return

### 1.2.2.4 Optimal Control Over 3 Day Horizon

Now let's use our functions for some analysis. Calculate the optimal battery operation strategy over 3 days starting on January 4, 2015.

In [ ]:
three_days = prepare_data_array(4,3)

#### 1.2.2.4.1 Original Model

In [ ]:
# YOUR SOLUTION HERE

#### 1.2.2.4.2 With Constraint Enforcing No Simultaneous Charging or Discharging

In [ ]:
# YOUR SOLUTION HERE

#### 1.2.2.4.3 Comparison

Write Python code below to compare the solutions. Print out any timesteps where the charge, discharge, and energy profiles are different by more than 10$^{-4}$ MW (or MWh).

In [ ]:
def compare_solutions(solution1,solution2):
    '''
    Function to compare outputs between the original model and model with disjuncutions
    
    Inputs:
    original: list of original charging/discharging data
    disjunction: list of disjunction model results charging/discharging data
    t: list of timesteps
    
    Action:
    prints same/different + timestep
    '''
    
    assert len(solution1) == len(solution2), "solution 1 and solution2 must be the same length"
    
    n = len(solution1)
    
    # YOUR SOLUTION HERE
    print("Done.")

In [ ]:
# Charge
compare_solutions(c_control_og, c_control_disj)

In [ ]:
# Discharge
compare_solutions(d_control_og, d_control_disj)

In [ ]:
# Energy
compare_solutions(E_control_og, E_control_disj)

## 1.2.3 Receding Horizon Control

We now have functions to build, solve, and analyze the battery problem for a single problem. Now we can consider receeding horizon control.

*Please ask questions during class. You do not need to turn anything in for this section.*

Nomenclature: Let $u^*_{i}$ represent the optimal control action from time $t=i$ to $t=i+1$. For the battery system, $u_{i} = [c_i, d_i]^{T}$ (charging and discharging).

Algorithm:
1. Consider time $t=m$. 
2. Compute the optimal control actions $u^*=[u^*_m, u^*_{m+1}, ..., u^*_{m+N-1}]^T$ from time $t=m$ until $t=N+m$ where $N$ is the length of the planning horizon. In our battery example, we considered $N = 24$.
3. Simulate the system for one timestep using control action $u^*_{m}$.
4. (Optional). Update the system model or forecasts using new observations.
5. Step $t=m+1$ and goto Step 2.

### 1.2.3.1 Write Pseudocode

Pseudocode is an important planning tool for complex computer programs. The idea is to write out the code on paper or a whiteboard using generic programming syntax. As an example, consider calculating the 3rd through 10th elements of the Fibonacci sequence:

```
# Governing equation where f[i] is the ith element of the sequence
# f[i] = f[i-1] + f[i-2]

# Algorithm:

# Previous previous number in the sequence, f[i-2]
n_2prev = 1

# Previous number in the sequence, f[i-1]
n_1prev = 1

# Set counter
j = 3

while j <= 10 do:
    # Apply Fib. formula
    n = n1_prev + n2_prev
    
    # Print to screen
    print(n)
    
    # Shift history.
    # Previous number becomes previous previous number
    n2prev = n1prev
    
    # Current number becomes previous number
    n1prev = n
    
    # Increment counter
    j = j + 1
```

Here we sketched out the main algorithm (with comments!) in generic syntax. We did not worry about the correct way to write a while loop in Python. Once we figure out the algorithm we can worry about those details.

Write pseudocode for the receding horizon control example on paper or a whiteboard. Your algorithm should include the following main steps:
1. Load price data from a text file
2. Loop over timesteps. Keep track of implemented control actions (charge and discharge), battery SOC, and net revenue.
3. Make a plot of battery SOC and control actions that were actually implemented.

Scan your pseudocode as a PDF and turn in via Gradescope.

### 1.2.3.2 Python Code

Tip: You should use the same `create_model` function throughout the assignment. In your model, define `price` to be a mutable parameter. This means we can just update the Pyomo model in the function below. We do not need to recreate it.

In [ ]:
def update_model(model, new_price_data, new_E0):
    """ Update the Pyomo model with new price data
    
    Arguments:
        model: pyomo model
        new_price_data: numpy array or list containing price data
        new_E0: (float) value for initial battery charge
    """
    
    # YOUR SOLUTION HERE

In [ ]:
def receding_horizon_simulation(price_data, simulation_length, num_hours, include_pbc=True,verbose=True):  
    '''
    Perform receding horizon simulation
    
    Inputs:
        price_data: numpy array price data for entire time period of interest
        simulation_length: number of total hours to evaluate
        num_hours: horizon length for each optimization problem
        include_pbc: boolean, include periodic boundary condition
        verbose: boolean, if true, print out message each hour
    
    Returns:
        c_control: charging decision implemented
        d_control: discharging decision implemented
        E_control: state-of-charge profile
        revenue_control: total revenue over simulation horizon
    
    '''
    
    # Check input data length
    assert len(price_data) >= simulation_length + num_hours, "Input price data is too short"
    
    assert simulation_length > 0, "Must evaluate more than 0 days"
    
    assert num_hours > 0, "Must consider planning horizon longer than 0 days"

    nS = simulation_length
        
    ## Preallocate vectors to store results
    c_control = np.empty(nS)
    d_control = np.empty(nS)
    E_control = np.empty(nS)
    revenue_control = np.empty(nS)

    # YOUR SOLUTION HERE

    return c_control, d_control, E_control, revenue_control

### 1.2.3.3 Simulate Performance Over 3 Days

Perform receding horizon control for the first 3 days in January 2015. Make a plot to show the implemented control action.

Tips:
* The implemented control action is a concationation of the first timestep for each optimization problem.
* How can you resuse the plot function above?

In [ ]:
three_days = prepare_data_array(0,4)

In [ ]:
# Tip: do not overwrite these results. We will use them later.
# If you reuse this code, change the variable names used to save the results.
c_control_pbc, d_control_pbc, E_control_pbc, revenue_control_pbc = receding_horizon_simulation(three_days, 24*3, 
                                                                               24, include_pbc=True,
                                                                              verbose=True)

In [ ]:
plot_solution(c_control_pbc, d_control_pbc, E_control_pbc)

### 1.2.3.4 Impact of Horizon Length

Repeat the sensitivity analysis for horizon lengths of N=2 through N=48. For each simulation, compute the following:
1. Revenue from first 30 days
2. Total energy purchased from the market during the first 30 days
3. Total energy sold to the market during the first 30 days
4. Total computation time

In [ ]:
# Study this cell to see how to time calculations in Python
import time

time_start = time.perf_counter()
first_month = prepare_data_array(0,34)
time_elapsed = (time.perf_counter() - time_start)

print("Elapsed Time",time_elapsed,"s")

In [ ]:
rev = []
charge = []
discharge = []
timer = []

# Tip: This will take a few minutes to finish. While developing your solutions, 
# only simulate the first few values of N. This will be helpful if you need to
# restart your kernel and rerun the cells. Once you are happy with your code
# for the entire assignment, you can set N to the values given here then
# restart and run all.
N = [2, 4, 6, 8, 12, 24, 36, 48]

# YOUR SOLUTION HERE

Make four well labeled plots to show these trends.

In [ ]:
plt.plot(N,rev,'o-',color="purple",linewidth=3)
plt.xlabel('N',fontsize=18)
plt.ylabel('Revenue, $',fontsize=18)
plt.grid(True)
plt.show()

In [ ]:
plt.plot(N,charge,'ro-',linewidth=3)
plt.xlabel('N',fontsize=18)
plt.ylabel('Charging, [MWh]',fontsize=18)
plt.grid(True)
plt.show()

In [ ]:
plt.plot(N,discharge,'go-',linewidth=3)
plt.xlabel('N',fontsize=18)
plt.ylabel('Discharging, [MWh]',fontsize=18)
plt.grid(True)
plt.show()

In [ ]:
plt.plot(N,timer,'ko-',linewidth=3)
plt.xlabel('N',fontsize=18)
plt.ylabel('Computational Time, [s]',fontsize=18)
plt.grid(True)
plt.show()

**Discussion** How long of a simulation horizon do you recommend? Discuss the trade-off between revenue and computation time.

### 1.2.3.5 Impact of Periodic Boundary Condition

We will now explore the impact of the periodic boundary constraint.

First, simulate the performance over 3 days **without enforcing the periodic boundary constraint**. Plot the implemented control action and calculate the total revenue. Use $N=4$.

In [ ]:
c_control2, d_control2, E_control2, revenue_control = receding_horizon_simulation(three_days, 24*3, 4,
                                                                                  include_pbc=False,verbose=True)

In [ ]:
plot_solution(c_control2, d_control2, E_control2)

Next, simulate **with the periodic boundary constraint** using $N=4$. Plot the results.

In [ ]:
c_control3, d_control3, E_control3, revenue_control = receding_horizon_simulation(three_days, 24*3, 4,
                                                                                  include_pbc=True,verbose=False)
plot_solution(c_control3, d_control3, E_control3)

Now let's compare.

In [ ]:
# Charge
compare_solutions(c_control2, c_control3)

In [ ]:
# Discharge
compare_solutions(d_control2, d_control3)

In [ ]:
# Energy
compare_solutions(E_control2, E_control3)

**Discussion** Compare the control actions and revenue with and without the periodic boundary constraint. How are they different (if at all)? Offer an explanation for any differences. (The purpose of this questions is to get you thinking about how to critically analyze the optimization results. This skill will be really important for the semester projects.)

Next, repeat the horizon length sensitivity analysis without the peridoic boundary constraint.

In [ ]:
# YOUR SOLUTION HERE

**Discussion** Compare these result to the analysis with periodic boundary constraints. Based on these results, make a recommendation to a battery operator.

## 1.2.4 Forecast Uncertainty (required for CBE 60499, optional for CBE 40499)

So far, we have assumed the battery operator can perfectly forecast the market price. This is not realistic. What is the impact of uncertainty?

Generate white noise with mean zero and standard deviation 5 \$/MWh using `np.random.normal()` and add this to the historical price. We will treat this as a price forecast. Simulate the first three days with $N=24$. Calculate how much less revenue you make with price uncertainty compared to the perfect information case. *Hint:* Each time you run the simulation, you will get a different answer. Repeat the analysis 10 times and record the average.

**Are you enrolled in CBE 60499** (are you a graduate student)? Please answer yes/no:

In [ ]:
revenue_unc_predicted = []
revenue_unc_actual = []

# Perform simulation with perfect information
c_control, d_control, E_control, revenue_control = receding_horizon_simulation(three_days, 
                                                                                       24*3, 24, include_pbc=True,
                                                                                  verbose=False)
# We can also calculate the revenue after the fact
rev_check = 0
for t in range(len(c_control)):
    # Why is this slightly different? In the instructor/TA solutions for receding_horizon_simulation,
    # we added a negative sign to d_control for plotting.
    rev_check -=  three_days[t]*(d_control[t] + c_control[t])

revenue_perfect_information = np.sum(revenue_control)
print("Revenue with perfect price forecast =",revenue_perfect_information,"=",rev_check,"$")

# Tip: normalize the revenue from the uncertain simulations by `revenue_perfect_information`
# This will make the interpretation much easier!

n = len(three_days)
nsim = 10

for i in range(nsim):

    # Add random noise to data
    three_days_with_noise = three_days + 5*np.random.normal(size=n)

    # YOUR SOLUTION HERE
    
print("Average Revenue =",np.mean(revenue_unc_actual)/revenue_perfect_information*100,"%")

**Discussion** Compare the results with your earlier analysis. How much less revenue do you get with Gaussian noise with standard deviation 5 $/MWh.

Next, make a plot showing the average revenue as a function of the standard deviation of the uncertainty. *Hint*: Normalize the revenue by dividing by the perfect information revenue (no uncertainty).

In [ ]:
# YOUR SOLUTION HERE

**Discussion** What is the impact with the larger uncertainty?

**Discussion** Why does uncertainty in the price decrease the revenue (on average)?

<!--NAVIGATION-->
< [1.1 60 Minutes to Pyomo: An Energy Storage Model Predictive Control Example](https://ndcbe.github.io/CBE60499/01.01-Pyomo-Nuts-and-Bolts.html) | [Contents](toc.html) | [Tag Index](tag_index.html) | [2.0 Optimization Modeling with Applications](https://ndcbe.github.io/CBE60499/02.00-Optimization-Modeling.html) ><p><a href="https://colab.research.google.com/github/ndcbe/CBE60499/blob/master/docs/01.02-Pyomo-Mini-Project.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://ndcbe.github.io/CBE60499/01.02-Pyomo-Mini-Project.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>